In [1]:
from sqlalchemy import create_engine, text
from sqlalchemy import text
import psycopg2
import psycopg2.extras
import json
import os
import pandas as pd
import requests
import ipywidgets as widgets
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

def display_image(response, w=200, h=300):
    return widgets.Image(value=response.content, format='jpg', width=w, height=h)

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

db, conn = pgconnect(credentials)

from sqlalchemy import text
conn.execute(text("set search_path to sa2project"))

Connected successfully.


In [2]:
conn.commit()
conn.execute(text("""
DROP TABLE IF EXISTS businesses CASCADE;
CREATE TABLE businesses(
   industry_code TEXT,
   industry_name TEXT,
   sa2_code TEXT,
   sa2_name TEXT,
   "0_to_50k_businesses" INTEGER NOT NULL,
   "50k_to_200k_businesses" integer NOT NULL,
   "200k_to_2m_businesses" integer NOT NULL,
   "2m_to_5m_businesses" integer NOT NULL,
   "5m_to_10m_businesses" integer NOT NULL,
   "10m_or_more_businesses" integer NOT NULL,
   total_businesses integer
);"""))
businessdata = pd.read_csv('businesses.csv')
businessdata.columns = map(str.lower, businessdata.columns)
businessdata.to_sql("businesses", con=conn, if_exists='append', index=False)

217

In [3]:
conn.commit()
conn.execute(text("""
DROP TABLE IF EXISTS income CASCADE;
CREATE TABLE income(
    sa2_code21 integer,
    sa2_name text,
    earners integer,
    median_age integer,
    median_income integer,
    mean_income integer
);"""))
incomedata = pd.read_csv('income.csv')
incomedata.columns = map(str.lower, incomedata.columns)
numeric_columns = ['earners', 'median_age', 'median_income', 'mean_income']
for col in numeric_columns:
    if col in incomedata.columns:
        incomedata[col] = pd.to_numeric(incomedata[col], errors='coerce')
incomedata.to_sql("income", con=conn, if_exists='append', index=False)

642

In [4]:
conn.commit()
conn.execute(text("""
DROP TABLE IF EXISTS population CASCADE;
CREATE TABLE population(
    sa2_code integer,
    sa2_name text,
    "0-4_people" INTEGER,
    "5-9_people" INTEGER,
    "10-14_people" INTEGER,
    "15-19_people" INTEGER,
    "20-24_people" INTEGER,
    "25-29_people" INTEGER,
    "30-34_people" INTEGER,
    "35-39_people" INTEGER,
    "40-44_people" INTEGER,
    "45-49_people" INTEGER,
    "50-54_people" INTEGER,
    "55-59_people" INTEGER,
    "60-64_people" INTEGER,
    "65-69_people" INTEGER,
    "70-74_people" INTEGER,
    "75-79_people" INTEGER,
    "80-84_people" INTEGER,
    "85-and-over_people" INTEGER,
    "total_people" INTEGER
);"""))
populationdata = pd.read_csv('population.csv')
populationdata.columns = map(str.lower, populationdata.columns)
populationdata.to_sql("population", con=conn, if_exists='append', index=False)

373

In [8]:
import geopandas as gpd
from shapely.geometry import Point
from sqlalchemy import text
conn.execute(text("SET search_path TO sa2project, public")) # Ensure schema context
conn.commit()

create_stops_table_sql = """
DROP TABLE IF EXISTS STOPS CASCADE;
CREATE TABLE STOPS (
    stop_id VARCHAR(20) PRIMARY KEY, -- Added PRIMARY KEY constraint
    stop_code VARCHAR(20),
    stop_name TEXT,
    location_type VARCHAR(5),
    parent_station VARCHAR(20),
    wheelchair_boarding INTEGER,
    platform_code VARCHAR(20),
    -- Added geometry column with Point type and SRID 4326 (WGS84 for lat/lon)
    geometry GEOMETRY(Point, 4326)
);
"""
try:
    conn.execute(text(create_stops_table_sql))
    conn.commit()

    # Load data using pandas and geopandas
    print("Reading Stops.txt...")
    stops_df = pd.read_csv("Stops.txt")
    stops_df.columns = map(str.lower, stops_df.columns) 
    stops_df = stops_df.replace({np.nan: None, "": None})
    stops_df['stop_lat'] = pd.to_numeric(stops_df['stop_lat'], errors='coerce')
    stops_df['stop_lon'] = pd.to_numeric(stops_df['stop_lon'], errors='coerce')

    original_count = len(stops_df)
    stops_df.dropna(subset=['stop_lat', 'stop_lon'], inplace=True)
    if len(stops_df) < original_count:
        print(f"Warning: Dropped {original_count - len(stops_df)} rows due to missing lat/lon.")

    print("Creating GeoDataFrame...")
    stops_gdf = gpd.GeoDataFrame(
        stops_df,
        geometry=gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat),
        crs="EPSG:4326" 
    )
    stops_gdf = stops_gdf.drop(columns=['stop_lat', 'stop_lon'])

    int_cols = ['wheelchair_boarding', 'location_type']
    for col in int_cols:
         if col in stops_gdf.columns:
            stops_gdf[col] = pd.to_numeric(stops_gdf[col], errors='coerce').astype('Int64')

    print("Writing STOPS data to PostGIS...")
    stops_gdf.to_postgis(
        name='stops',       # Table name (lowercase recommended)
        con=conn,           # Database connection engine
        schema='sa2project',# Target schema
        if_exists='append', # Append data to the existing (now empty) table
        index=False         # Don't write the DataFrame index
    )
    conn.commit() 
    index_sql = "CREATE INDEX idx_stops_geom ON stops USING gist(geometry);"
    conn.execute(text(index_sql))
    conn.commit()

except FileNotFoundError:
    print("ERROR: Stops.txt not found. Please check the path.")
except Exception as e:
    conn.rollback() # Rollback on error
    print(f"An error occurred during STOPS table processing: {e}")

Creating STOPS table with geometry...
STOPS table created successfully with geometry column.
Reading Stops.txt...
Creating GeoDataFrame...
Writing STOPS data to PostGIS...
Successfully wrote STOPS data to PostGIS.
Creating spatial index on stops geometry...
Spatial index created successfully.


In [6]:
# ------------- Task 1: Load SA2 Boundaries -------------
print("Loading SA2 Boundaries shapefile...")
try:
    sa2_gdf = gpd.read_file('SA2/SA2_2021_AUST_GDA2020.shp')

    sa2_gdf.columns = map(str.lower, sa2_gdf.columns)
    if 'geometry' not in sa2_gdf.columns and 'geom' in sa2_gdf.columns:
         sa2_gdf = sa2_gdf.rename(columns={'geom': 'geometry'})
    elif 'geometry' not in sa2_gdf.columns:
         print("ERROR: Geometry column not found in SA2 shapefile.")
    conn.execute(text("SET search_path TO sa2project, public")) 
    sa2_gdf.to_postgis(
        name='sa2_boundaries',      # Choose a table name
        con=conn,                  # Your existing database connection engine
        schema='sa2project',       # Specify the schema
        if_exists='replace',       # Replace table if it already exists
        index=False                # Don't write the GeoDataFrame index as a column
    )
    conn.commit() # Commit the transaction
    print("Successfully wrote SA2 boundaries to PostGIS table 'sa2project.sa2_boundaries'.")

except FileNotFoundError:
    print("ERROR: SA2 shapefile not found. Please check the path 'SA2/SA2_2021_AUST_GDA2020.shp'.")
except Exception as e:
    conn.rollback()
    print(f"An error occurred during SA2 shapefile loading: {e}")

Loading SA2 Boundaries shapefile...
Successfully wrote SA2 boundaries to PostGIS table 'sa2project.sa2_boundaries'.


c

In [7]:
sa4data = gpd.read_file("sa4/SA4_2021_AUST_GDA2020.dbf")
pd.set_option('display.max_rows', None)
filtered = sa4data[sa4data["GCC_NAME21"] == "Greater Sydney"] 
filtered

,SA4_CODE21,SA4_NAME21,CHG_FLAG21,CHG_LBL21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
1,102,Central Coast,0,No change,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,1681.0088,http://linked.data.gov.au/dataset/asgsed3/SA4/102,"MULTIPOLYGON (((151.31497 -33.55578, 151.31496..."
14,115,Sydney - Baulkham Hills and Hawkesbury,0,No change,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,3251.4981,http://linked.data.gov.au/dataset/asgsed3/SA4/115,"MULTIPOLYGON (((151.15205 -33.52766, 151.15205..."
15,116,Sydney - Blacktown,0,No change,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,240.8835,http://linked.data.gov.au/dataset/asgsed3/SA4/116,"POLYGON ((150.87089 -33.82385, 150.87078 -33.8..."
16,117,Sydney - City and Inner South,0,No change,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,66.1035,http://linked.data.gov.au/dataset/asgsed3/SA4/117,"MULTIPOLYGON (((151.19474 -33.85326, 151.19501..."
17,118,Sydney - Eastern Suburbs,0,No change,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,57.7342,http://linked.data.gov.au/dataset/asgsed3/SA4/118,"MULTIPOLYGON (((151.26545 -33.9263, 151.26533 ..."
18,119,Sydney - Inner South West,0,No change,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,163.9272,http://linked.data.gov.au/dataset/asgsed3/SA4/119,"POLYGON ((151.09654 -33.99094, 151.09654 -33.9..."
19,120,Sydney - Inner West,0,No change,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,64.5491,http://linked.data.gov.au/dataset/asgsed3/SA4/120,"MULTIPOLYGON (((151.17407 -33.84586, 151.17377..."
20,121,Sydney - North Sydney and Hornsby,0,No change,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,275.0994,http://linked.data.gov.au/dataset/asgsed3/SA4/121,"MULTIPOLYGON (((151.17864 -33.5155, 151.17861 ..."
21,122,Sydney - Northern Beaches,0,No change,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,254.2072,http://linked.data.gov.au/dataset/asgsed3/SA4/122,"MULTIPOLYGON (((151.32886 -33.66075, 151.32885..."
22,123,Sydney - Outer South West,0,No change,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,1277.2421,http://linked.data.gov.au/dataset/asgsed3/SA4/123,"POLYGON ((150.82039 -33.97522, 150.8204 -33.97..."


In [8]:
def poi_in_bounding_box(xmin, ymin, xmax, ymax, filters={}):
    baseURL = 'https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer/0/query'
    params = {
        'geometry': f'{xmin},{ymin},{xmax},{ymax}',
        'geometryType': 'esriGeometryEnvelope',
        'inSR': '4326',
        'spatialRel': 'esriSpatialRelIntersects',
        'outFields': '*',
        'returnGeometry': 'true',
        'f': 'json',
        'where': ' AND '.join([f"{k}='{v}'" for k, v in filters.items()]) if filters else '1=1'
    }
    response = requests.get(baseURL, params=params)
    return json.loads(response.text)['features']

In [9]:
import time

sa2 = gpd.read_file('SA2/SA2_2021_AUST_GDA2020.shp')
selected_sa4 = 'Sydney - Blacktown'
sa2_in_sa4 = sa2[sa2['SA4_NAME21'] == selected_sa4]

pois = []
for idx, row in sa2_in_sa4.iterrows():
    xmin, ymin, xmax, ymax = row['geometry'].bounds  
    # Create a different variable name instead of reusing 'pois'
    poi_results = poi_in_bounding_box(xmin, ymin, xmax, ymax)

    for poi in poi_results:
        poi['SA2_NAME21'] = row['SA2_NAME21']  
        pois.append(poi)
    time.sleep(1)
pois

[{'attributes': {'objectid': 1836,
   'topoid': 500245990,
   'poigroup': 3,
   'poitype': 'Park',
   'poiname': 'BREWONGLE WALKWAY',
   'poilabel': 'BREWONGLE WALKWAY',
   'poilabeltype': 'NAMED',
   'poialtlabel': None,
   'poisourcefeatureoid': 61,
   'accesscontrol': 1,
   'startdate': 1285588392000,
   'enddate': 32503680000000,
   'lastupdate': 1285588392535,
   'msoid': 89017,
   'centroidid': None,
   'shapeuuid': '0b0df18d-085e-30f0-b593-ad0595c63321',
   'changetype': 'I',
   'processstate': None,
   'urbanity': 'U'},
  'geometry': {'x': 150.89843351955943, 'y': -33.77774757918883},
  'SA2_NAME21': 'Blacktown (East) - Kings Park'},
 {'attributes': {'objectid': 1837,
   'topoid': 500246012,
   'poigroup': 3,
   'poitype': 'Park',
   'poiname': None,
   'poilabel': 'Park',
   'poilabeltype': 'GENERIC',
   'poialtlabel': None,
   'poisourcefeatureoid': 61,
   'accesscontrol': 1,
   'startdate': 1285588392000,
   'enddate': 32503680000000,
   'lastupdate': 1285588392535,
   'msoi

In [10]:
conn.commit()
create_table_query = """
DROP TABLE IF EXISTS PointsOfInterest;
CREATE TABLE PointsOfInterest (
    objectid BIGINT PRIMARY KEY,
    topoid BIGINT,
    poigroup TEXT,
    poitype TEXT,
    poiname TEXT,
    poilabel TEXT,
    poilabeltype TEXT,
    poialtlabel TEXT,
    poisourcefeatureoid BIGINT,
    accesscontrol TEXT,
    startdate bigint,
    enddate bigint,
    lastupdate bigint,
    msoid BIGINT,
    centroidid BIGINT,
    shapeuuid TEXT,
    changetype TEXT,
    processstate TEXT,
    urbanity TEXT,
    x FLOAT,
    y FLOAT,
    sa2_name21 TEXT
);
"""

# Execute the drop and create operations
conn.execute(text(create_table_query))

# Insert data query
insert_query = """
INSERT INTO PointsOfInterest (
    objectid, topoid, poigroup, poitype, poiname, poilabel, poilabeltype, poialtlabel,
    poisourcefeatureoid, accesscontrol, startdate, enddate, lastupdate,
    msoid, centroidid, shapeuuid, changetype, processstate, urbanity,
    x, y, sa2_name21
) VALUES (
    :objectid, :topoid, :poigroup, :poitype, :poiname, :poilabel, :poilabeltype, :poialtlabel,
    :poisourcefeatureoid, :accesscontrol, :startdate, :enddate, :lastupdate,
    :msoid, :centroidid, :shapeuuid, :changetype, :processstate, :urbanity,
    :x, :y, :SA2_NAME21
) ON CONFLICT (objectid) DO NOTHING;
"""

# Insert data
for poi in pois:
    attributes = poi.get('attributes', {})
    geometry = poi.get('geometry', {})
    data = {
        'objectid': attributes.get('objectid'),
        'topoid': attributes.get('topoid'),
        'poigroup': attributes.get('poigroup'),
        'poitype': attributes.get('poitype'),
        'poiname': attributes.get('poiname'),
        'poilabel': attributes.get('poilabel'),
        'poilabeltype': attributes.get('poilabeltype'),
        'poialtlabel': attributes.get('poialtlabel'),
        'poisourcefeatureoid': attributes.get('poisourcefeatureoid'),
        'accesscontrol': attributes.get('accesscontrol'),
        'startdate': attributes.get('startdate'),
        'enddate': attributes.get('enddate'),
        'lastupdate': attributes.get('lastupdate'),
        'msoid': attributes.get('msoid'),
        'centroidid': attributes.get('centroidid'),
        'shapeuuid': attributes.get('shapeuuid'),
        'changetype': attributes.get('changetype'),
        'processstate': attributes.get('processstate'),
        'urbanity': attributes.get('urbanity'),
        'x': geometry.get('x'),
        'y': geometry.get('y'),
        'SA2_NAME21': poi.get('SA2_NAME21')
    }
    conn.execute(text(insert_query), data)

print("Table created/recreated and data inserted successfully.")

Table created/recreated and data inserted successfully.


In [11]:
conn.execute(text("SET search_path TO sa2project, public"))
conn.commit()

In [9]:
import geopandas as gpd
import pandas as pd
from sqlalchemy import text
import os

catchment_dir = 'Catchments/catchments/'

# List the expected shapefiles - ADJUST FILENAMES if they are different!
catchment_files = {
    'Primary': os.path.join(catchment_dir, 'catchments_primary.shp'),
    'Secondary': os.path.join(catchment_dir, 'catchments_secondary.shp'),
    'Future': os.path.join(catchment_dir, 'catchments_future.shp') # Adjust if filename differs
}

all_catchments_list = [] # To store individual GeoDataFrames
base_crs = None # To store the CRS of the first file read

print("Loading and combining school catchment shapefiles...")

# Loop through the defined files
for catchment_type, file_path in catchment_files.items():
    if not os.path.exists(file_path):
        print(f"Warning: Shapefile not found at {file_path}. Skipping.")
        continue
    try:
        gdf = gpd.read_file(file_path)

        gdf['catchment_type'] = catchment_type

        if gdf.crs is None:
            print(f"Warning: CRS not set for {file_path}. Assuming GDA2020 (EPSG:7844). Set explicitly if different.")
            gdf.set_crs(epsg=7844, inplace=True) # Adjust EPSG if needed (e.g., 4283 for GDA94)

        if base_crs is None:
            base_crs = gdf.crs # Set the CRS from the first file
            print(f"Detected base CRS: {base_crs}")
        elif gdf.crs != base_crs:
            print(f"Warning: CRS mismatch for {file_path} ({gdf.crs}). Transforming to base CRS ({base_crs}).")
            gdf = gdf.to_crs(base_crs)
        # -----------------------------------------------------

        all_catchments_list.append(gdf)
        print(f"Successfully read {len(gdf)} features from {file_path}")

    except Exception as e:
        print(f"ERROR: Could not read or process {file_path}. Error: {e}")

if all_catchments_list:
    combined_catchments_gdf = pd.concat(all_catchments_list, ignore_index=True)
    print(f"Combined total of {len(combined_catchments_gdf)} catchment features.")

    combined_catchments_gdf.columns = map(str.lower, combined_catchments_gdf.columns)
    if 'geometry' not in combined_catchments_gdf.columns and 'geom' in combined_catchments_gdf.columns:
         combined_catchments_gdf = combined_catchments_gdf.rename(columns={'geom': 'geometry'})
    elif 'geometry' not in combined_catchments_gdf.columns:
         print("ERROR: Geometry column not found in combined catchment data. Cannot write to PostGIS.")
         combined_catchments_gdf = None 
    if combined_catchments_gdf is not None:
        try:
            print("Writing combined school catchments to PostGIS table 'sa2project.schoolcatchments'...")
            conn.execute(text("SET search_path TO sa2project, public"))
            conn.commit()

            combined_catchments_gdf.to_postgis(
                name='schoolcatchments',      # Table name
                con=conn,                     # Database connection engine
                schema='sa2project',          # Target schema
                if_exists='replace',          # Replace the table if it exists
                index=False                   # Don't write the GeoDataFrame index
            )
            conn.commit() # Commit transaction
            print("Successfully wrote combined data to PostGIS.")

            # Optional: Add a spatial index (HIGHLY RECOMMENDED)
            print("Creating spatial index on schoolcatchments geometry...")
            index_sql = "CREATE INDEX idx_schoolcatchments_geom ON sa2project.schoolcatchments USING gist(geometry);"
            conn.execute(text("DROP INDEX IF EXISTS sa2project.idx_schoolcatchments_geom;"))
            conn.execute(text(index_sql))
            conn.commit()
            print("Spatial index created successfully.")

        except Exception as e:
            conn.rollback() 
            print(f"An error occurred writing combined catchments to PostGIS: {e}")
else:
    print("No school catchment data loaded. Cannot write to PostGIS.")

# ----------------------------------------------------------

Loading and combining school catchment shapefiles...
Detected base CRS: EPSG:4283
Successfully read 1662 features from Catchments/catchments/catchments_primary.shp
Successfully read 436 features from Catchments/catchments/catchments_secondary.shp
Successfully read 30 features from Catchments/catchments/catchments_future.shp
Combined total of 2128 catchment features.
Writing combined school catchments to PostGIS table 'sa2project.schoolcatchments'...
Successfully wrote combined data to PostGIS.
Creating spatial index on schoolcatchments geometry...
Spatial index created successfully.


In [13]:
def businesses_per_1000():
    
    sql = """
    SELECT B.sa2_name, B.total_businesses, P.total_people
    FROM Businesses B JOIN Population P ON B.sa2_name = P.sa2_name
    WHERE B.sa2_name in (SELECT distinct(sa2_name21) FROM PointsOfInterest) 
    and B.industry_name = 'Retail Trade'
    AND P.total_people > 100;
    """
    businesses = query(conn,sql)

    businesses_per_1000 = {}
    for idx, row in businesses.iterrows():
        sa2_name = row['sa2_name']
        total_businesses = row['total_businesses']
        total_people = row['total_people']

        per_1000 = (total_businesses / total_people) * 1000
        businesses_per_1000[sa2_name] = per_1000

    return businesses_per_1000 # returns Dictionary with key-value as sa2_name : number of businesses per 1000 people

def stop_counts(target_sa2_names):
    if not target_sa2_names:
        print("Warning: No target SA2 names provided for stop_counts.")
        return {}

    sa2_names_tuple = tuple(target_sa2_names)

    sql = """
    SELECT
        sa2.sa2_name21,
        COALESCE(COUNT(st.stop_id), 0) AS stop_count
    FROM
        sa2project.sa2_boundaries sa2 
    LEFT JOIN
        sa2project.stops st
        ON ST_Contains(ST_Transform(sa2.geometry, 4326), st.geometry)
    WHERE
        sa2.sa2_name21 IN :sa2_names_tuple 
    GROUP BY
        sa2.sa2_name21;
    """

    print("Executing PostGIS query to count stops per SA2...")
    stop_counts_df = query(conn, text(sql), args={'sa2_names_tuple': sa2_names_tuple}, df=True)

    if stop_counts_df is None or stop_counts_df.empty:
        print("Warning: stop_counts query returned no results.")
        return {name: 0 for name in target_sa2_names}

    stop_counts_dict = dict(zip(stop_counts_df['sa2_name21'], stop_counts_df['stop_count']))
    for name in target_sa2_names:
        if name not in stop_counts_dict:
            stop_counts_dict[name] = 0

    print("Stop counts calculated using PostGIS.")
    return stop_counts_dict

sa2_names_in_selected_sa4 = sa2_in_sa4['SA2_NAME21'].tolist()

def schools_per_1000(target_sa2_names, conn):
    if not target_sa2_names:
        print("Warning: No target SA2 names provided for schools_per_1000.")
        return {}

    sa2_names_tuple = tuple(target_sa2_names)

    sql = """
    WITH YoungPop AS (
        SELECT
            sa2_name,
            ("0-4_people" + "5-9_people" + "10-14_people" + "15-19_people") AS total_young_people
        FROM sa2project.population
        WHERE sa2_name IN :sa2_names_tuple
          AND ("0-4_people" + "5-9_people" + "10-14_people" + "15-19_people") >= 100
    ),
    CatchmentCounts AS (
        SELECT
            sa2.sa2_name21,
            COUNT(sc.ogc_fid) AS catchment_count
        FROM
            sa2project.sa2_boundaries sa2
        JOIN
            sa2project.schoolcatchments sc
            ON ST_Intersects(
                ST_Transform(sa2.geometry, 4326),
                ST_Transform(sc.geometry, 4326)
            )
        WHERE
            sa2.sa2_name21 IN :sa2_names_tuple -- Ensure we only process target SA2s
        GROUP BY
            sa2.sa2_name21
    )
    SELECT
        yp.sa2_name,
        (COALESCE(cc.catchment_count, 0) * 1000.0 / yp.total_young_people) AS schools_per_1000
    FROM
        YoungPop yp
    LEFT JOIN -- Use LEFT JOIN to keep all SA2s from YoungPop (those with >= 100 young people)
        CatchmentCounts cc ON yp.sa2_name = cc.sa2_name21; -- Match SA2 names
    """

    print("Executing PostGIS query to calculate schools per 1000 young people...")
    try:
        schools_ratio_df = query(conn, text(sql), args={'sa2_names_tuple': sa2_names_tuple}, df=True)

        if schools_ratio_df is None or schools_ratio_df.empty:
            print("Warning: schools_per_1000 query returned no results.")
            return {name: 0.0 for name in target_sa2_names}
        schools_ratio_dict = dict(zip(schools_ratio_df['sa2_name'], schools_ratio_df['schools_per_1000']))

        final_dict = {}
        young_pop_sa2s = set(query(conn, text("SELECT sa2_name FROM YoungPop"), df=True)['sa2_name']) # Re-query YoungPop SA2s
        for name in target_sa2_names:
             if name in young_pop_sa2s: # Only include SA2s that met the >=100 young people criteria
                final_dict[name] = schools_ratio_dict.get(name, 0.0) # Get score or default to 0

        print("Schools per 1000 young people calculated using PostGIS.")
        return final_dict

    except Exception as e:
        print(f"An error occurred during schools_per_1000 query: {e}")
        return {}


def POI_counts():
    sql = """
    SELECT sa2_name21, Count(*) as Poi_count
    FROM PointsOfInterest
    WHERE poigroup = '8'
    GROUP BY sa2_name21
    """
    poi_counts = query(conn,sql)

    if poi_counts is None or poi_counts.empty:
        print("Warning: POI_counts query returned no results.")
        return {}

    poi_dict = dict(zip(poi_counts['sa2_name21'], poi_counts['poi_count']))
    return poi_dict

In [14]:
conn.commit()
businesses_per_1000_people = businesses_per_1000()
transport_stop_counts = stop_counts(sa2_names_in_selected_sa4)
school_per_1000_people = schools_per_1000(sa2_in_sa4)
POI = POI_counts()

In [16]:
def z_score(data_dict):
    series = pd.Series(data_dict)
    z_series = (series - series.mean()) / series.std()
    return z_series.to_dict()

print(z_score(businesses_per_1000_people))
print(z_score(transport_stop_counts))
print(z_score(school_per_1000_people))
print(z_score(POI))

{'Blacktown (East) - Kings Park': 2.071190415555226, 'Blacktown (North) - Marayong': -0.43779635891160984, 'Doonside - Woodcroft': -0.43779635891160984, 'Lalor Park - Kings Langley': -0.43779635891160984, 'Blacktown - South': -0.43779635891160984, 'Blacktown - West': -0.43779635891160984, 'Seven Hills - Prospect': -0.43779635891160984, 'Toongabbie - West': -0.43779635891160984, 'Glenwood': -0.43779635891160984, 'Acacia Gardens': -0.43779635891160984, 'Quakers Hill': -0.43779635891160984, 'Kellyville Ridge - The Ponds': -0.43779635891160984, 'Marsden Park - Shanes Park': 2.3192649584898466, 'Riverstone': 2.662484889480276, 'Schofields (West) - Colebee': -0.43779635891160984, 'Schofields - East': 1.2651905557952392, 'Stanhope Gardens - Parklea': -0.43779635891160984, 'Bidwill - Hebersham - Emerton': -0.43779635891160984, 'Glendenning - Dean Park': -0.43779635891160984, 'Hassall Grove - Plumpton': -0.43779635891160984, 'Lethbridge Park - Tregear': -0.43779635891160984, 'Mount Druitt - Wha

In [17]:
import numpy as np
def Score(x):
    return 1 / (1 + np.exp(-x))

def combined_resource_score(business_z, stop_z, school_z, poi_z):
    final_scores = {}
    all_sa2s = set(business_z) & set(stop_z) & set(school_z) & set(poi_z)

    for sa2 in all_sa2s:
        total_z = business_z[sa2] + stop_z[sa2] + school_z[sa2] + poi_z[sa2]
        final_scores[sa2] = Score(total_z) 
        # final_scores[sa2] = round(Score(total_z), 3) if we want to round to 3 d.p

    return final_scores

businesses_z_score = z_score(businesses_per_1000_people)
stops_z_score = z_score(transport_stop_counts)
schools_z_score = z_score(school_per_1000_people)
POI_z_score = z_score(POI)

scores = combined_resource_score(businesses_z_score, stops_z_score, schools_z_score, POI_z_score)
print(scores)


{'Quakers Hill': np.float64(0.3607520258717122), 'Blacktown (East) - Kings Park': np.float64(0.9999850386102371), 'Doonside - Woodcroft': np.float64(0.23856635055454292), 'Lethbridge Park - Tregear': np.float64(0.20987370680905412), 'Riverstone': np.float64(0.9958914044135062), 'Rooty Hill - Minchinbury': np.float64(0.1756824998683663), 'Glendenning - Dean Park': np.float64(0.09186791288640288), 'Hassall Grove - Plumpton': np.float64(0.2622729514025215), 'Schofields - East': np.float64(0.294564357175185), 'Seven Hills - Prospect': np.float64(0.41089728714403534), 'Lalor Park - Kings Langley': np.float64(0.9406365169282493), 'Schofields (West) - Colebee': np.float64(0.035328590080536175), 'Marsden Park - Shanes Park': np.float64(0.9854439005930826), 'Glenwood': np.float64(0.1257212044186272), 'Blacktown - West': np.float64(0.05298711104111019), 'Blacktown - South': np.float64(0.08829003121174904), 'Kellyville Ridge - The Ponds': np.float64(0.318914538680193), 'Blacktown (North) - Marayo